# EELS analysis of perovskite oxides using eXSpy

This tutorial shows the various functionalities in eXSpy which is used to analyse Electron Energy Loss Spectroscopy data, using EELS datasets from a perovskite oxide heterostructure.

It assumes some knowledge on how to use HyperSpy, like loading datasets and how the basic signals work.

This notebook requires **HyperSpy 2.0.0** or later. In addition **eXSpy**, which has the EELS and EDX specific parts of HyperSpy: https://hyperspy.org/exspy/

## Author

7/6/2016 Magnus Nord - Developed for HyperSpy workshop at Scandem conference 2016

## Changes

* 3/8/2016 Updated for HyperSpy 1.1. Added note about Gatan Digital Micrograph GOS.
* 20/7/2019 Katherine MacArthur - Checked for Hyperspy 1.5.1 and commented out sections requiring Gatan GOS files.
* 30/7/2019 Magnus Nord - Minor text improvements for M&M19 short course
* 9/3/2024 Magnus Nord - Updated to work with HyperSpy 2.0.0
* 1/6/2025 Eric Prestat - reworked for the ePSIC HyperSpy workshop 2025

## Table of contents

1. <a href='#spec_and_data'> Specimen & Data</a>
2. <a href='#loading_and_aligning_data'> Loading and aligning data</a>
3. <a href='#fourier_ratio_deconvolution'> Fourier ratio deconvolution and peak fitting</a>
4. <a href='#fine_structure_fitting'> Fine structure fitting with convolution</a>
5. <a href='#visualise_fitting_results'> Visualise fitting results</a>

# <a id='spec_and_data'></a>1. Specimen & Data

The data was acquired on a Jeol ARM200cF using a Gatan Quantum ER with DualEELS capabilities.

The data itself is from La$_{0.7}$Sr$_{0.3}$MnO$_3$ thin films deposited on SrTiO$_3$. In the fine structure example parts of the film has been exposed to a very long electron beam exposure, inducing oxygen vacancies.

The datasets has been binned to reduce the file size and processing time.

# <a id='loading_and_aligning_data'></a> 2. Loading and aligning data

In [1]:
%matplotlib qt
import hyperspy.api as hs

Here we take a look at a linescan from a La$_{0.7}$Sr$_{0.3}$MnO$_3$ thin film, where parts of the film has been bombarded with the electron beam for an extended time.

We start by loading the core-loss dataset stored in `datasets/LSMO_linescan.hspy`

In [2]:
# open the "datasets/LSMO_linescan.hspy" file
s = hs.load("datasets/LSMO_linescan.hspy")

Plot the signal, and use the red line in the navigation plot to explore the data. There is clearly something going on in the middle on both the oxygen and the manganese edges. In addition, there are some thickness changes during the line scan.

In [3]:
# Plot the signal
s.plot()

Using the low loss signal, we make sure the energy scale is properly calibrated: `datasets/LSMO_linescan_low_loss.hspy`

In [4]:
# Open the "datasets/LSMO_linescan_low_loss.hspy"
s_ll = hs.load("datasets/LSMO_linescan_low_loss.hspy")

We plot the low loss signal.
`autoscale=""` can be used to be disable resetting the zoom when changing the navigation coordinate.

In [5]:
# Plot the signal setting the `autoscale` parameter to disable autoscale
s_ll.plot(autoscale="")

The zero loss peak is not well aligned at 0 eV energy loss. We align both signals (low-loss and core-loss) by measuring the energy shift on the zero loss peak and applying the shifts to both signals.

In [6]:
# align both signal (`s` and `s_ll`) using `align_zero_loss_peak`.
s_ll.align_zero_loss_peak(also_align=[s])

Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	3.23
std:	0.249

min:	3
Q1:	3
median:	3
Q3:	3.5
max:	3.5


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

The plotted signal has updated and we can checked that the zero loss peak has been shifted to 0 energy loss.

We can also calculate the relative thickness using the low loss, with the [`estimate_thickness`](https://hyperspy.org/exspy/reference/signals.html#exspy.signals.EELSSpectrum.estimate_thickness) method. We'll have to specify the end of the zero loss beam, which for cold field emissions guns 3.0 eV seems to work well.

In [7]:
# Use the `estimate_thickness` method' specify the energy threshold
s_thickness = s_ll.estimate_thickness(threshold=3.0)

Computing the thickness without taking into account the effect of the limited collection angle, what usually leads to underestimating the thickness. To perform the angular corrections you must provide the density of the material.
Computing the relative thickness. To compute the absolute thickness provide the `mean_free_path` and/or the `density`


It would also be possible to use hyperspy to determine the threshold itself using:
    
    th = s_ll.estimate_elastic_scattering_threshold()
    s_ll.estimate_thickness(threshold=th)

Plotting this gives the relative thickness and, as expected, there is an increase towards the end of the line scan

In [8]:
# Plot the thickness
s_thickness.T.plot()

# <a id='fourier_ratio_deconvolution'></a> 3. Fourier ratio deconvolution and peak fitting

## 3.1 Fourier ratio deconvolution
Lets take a closer look at the oxygen-K edge, firstly by removing the plasmon background with `remove_background`. Note: this will overwrite the `s` spectrum with the cropped one. 

In [9]:
# Remove the background using a signal range of (490, 520)
s_bg_removed = s.remove_background(signal_range=(490, 520))
# Plot the background subtracted signal
s_bg_removed.plot()

This makes it much easier to compare the different positions. Pressing 'e' with the spectrum window highlighted gives a second spectrum picker, which can be moved independently of the first one.

We can then do Fourier ratio deconvolution to remove the effects of plural scattering using the [`fourier_ratio_deconvolution`](https://hyperspy.org/exspy/reference/signals.html#exspy.signals.EELSSpectrum.fourier_ratio_deconvolution) method:

In [10]:
# Use the `fourier_ratio_deconvolution` method to perform Fourier ratio deconvolution
s_deconvolved = s_bg_removed.fourier_ratio_deconvolution(s_ll)

  0%|          | 0/2 [00:00<?, ?it/s]

Plotting this, we see that the thickness effects has been greatly reduced towards the end of the line scan

In [11]:
# Plot the deconvolved signal
s_deconvolved.plot()

For visualisation purposes, we can create a "meaningless" complex signal as a convenience to compare before and deconvolution, with
the real (plotted in red) and imaginary (plotted in blue) part corresponding to the before and after deconvolution, respectively.

In [12]:
# Create a "meaningless but convenient" complex signal to compare the signal before and after deconvolution 
comparison = s_bg_removed + s_deconvolved * 1j
# Plot the comparison
comparison.plot()

## 3.2 Peak fitting

Since these peaks we are trying to fit overlapping strongly, the fit is highly susceptible to fall into a local minimum during fit optimisation that doesn't describe well the experimental data. To avoid this issue, we constrain (_guide_) the optimisation by using bounded fitting and setting the bounds ([`bmin` and `bmax`](https://hyperspy.org/hyperspy-doc/current/reference/base_classes/model/parameter.html#hyperspy.component.Parameter)) of the `sigma` parameter of the [`Gaussian`](https://hyperspy.org/hyperspy-doc/current/reference/api.model/components1D.html#hyperspy.api.model.components1D.Gaussian) components.

The fitting approach consists in the following steps:
1. set bounding: `bmin = 0.75` and `bmax = 3.5` for the `sigma` parameters. By default the `bmin` of the `A` parameter is set to 0. We could also contrain the `centre` parameter but the fit is satisfactionary without adding this constraint
2. fit each component individually by specifying a fitting range. The order of the component fitting matters in this case: we start with the 2nd peak, then the 3rd and fit the 1st peak last. We choose to fit the first peak last because this peak is small and it overlaps strongly with the 2nd peak
3. Fix the `centre` parameter
4. refine the fit by fitting all components together

Alternative approaches could be considered, for example, setting the bounds to the `centre` parameter and/or avoid fixing the centre at a later could be worth trying.

As a convenience, we start by cropping the signal in energy to the region of interest - this is not necessary but it makes visualisation slightly easier.

In [13]:
# Use the `crop_signal` method to crop to the energy region of (500, 570)
s_deconvolved.crop_signal(500., 560.)

We create a model without a background because it was already removed before the deconvolution

In [14]:
# Create a model without background
m = s_deconvolved.create_model(auto_background=False)

In [15]:
# Plot the model and its components individually
m.plot(True)

We will fit three Gaussian functions to the edge. We create these componenents and add them to the model.

In [16]:
g1 = hs.model.components1D.Gaussian()
g1.name = "O_K_peak1"
g2 = hs.model.components1D.Gaussian()
g2.name = "O_K_peak2"
g3 = hs.model.components1D.Gaussian()
g3.name = "O_K_peak3"

In [17]:
# Add these components to the model
m.extend([g1, g2, g3])

Step 1: set the bounds of the `sigma` parameter

In [18]:
# Set the bmin and bmax of the `sigma` parameter of all Gaussian components
for g in [g1, g2, g3]:
    g.sigma.bmin = 0.75
    g.sigma.bmax = 3.5

Now we fit the components individually, starting from the 2nd peak as described previously

Step 2: fit each component individually

In [19]:
# Use the `fit_component` method of the model to fit the 2nd component
# specify signal range (528., 532.); fit all navigation coordinates; Use bounded fit
m.fit_component(g2, signal_range=(528., 532.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

In [20]:
# Same as last cell but for 3rd component; use signal range (535., 540.)
m.fit_component(g3, signal_range=(535., 540.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

In [21]:
# Same as last cell but for 1st component; use signal range (523., 526.)
m.fit_component(g1, signal_range=(523., 526.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

Step 3: fix the `centre` parameter of the three Gaussian components

In [22]:
# Use the `set_parameters_not_free` method of the model  
m.set_parameters_not_free(parameter_name_list=["centre"])

Fit for the current position to check that the fit converge to a satisfactory minimum; a few positions can be checked

In [23]:
# Make sure to use bounded fit
m.fit(bounded=True)

 message: 
 success: True
  status: 1
       x: [ 5.739e+05  1.309e+00  1.540e+06  1.773e+00  2.833e+06
            3.500e+00]
     nit: 9
   covar: [[ 1.781e+01  2.948e-05 ...  2.594e+00  0.000e+00]
           [ 2.948e-05  1.147e-10 ...  5.636e-06  0.000e+00]
           ...
           [ 2.594e+00  5.636e-06 ...  2.675e+01  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]]
  perror: [ 4.220e+00  1.071e-05  4.922e+00  6.625e-06  5.172e+00
            0.000e+00]
    nfev: 58
     dof: 114
   fnorm: 257194712132.09842

Once satisfied with current fit, fit for all navigation positions

In [24]:
# use `multifit` with bounding
m.multifit(bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

# <a id='fine_structure_fitting'></a> 4. Fine structure fitting with convolution


In this section, we will use a different approach: instead of performing a deconvolution before the fitting, we will convolve the mode during fitting. The second approach is expected to provide more accurate results and be more robust because of the following reasons:
- convolution is numerical more stable than deconvolution accross different experimental dataset, mostly because deconvolution is not a trivial problem to solve numerically: it uses iterative approaches and can also introduce artefacts.
- Fitting background and edges together usually better describe the experimental data.

We reload and align the data. Similarly as in the previous section, we start by cropping a region of the O-K edge, for example 450 to 590 eV as a convenience - here, we will use a wider signal range to fit the background and edges.

In [25]:
s2 = hs.load("datasets/LSMO_linescan.hspy")
s2_ll = hs.load("datasets/LSMO_linescan_low_loss.hspy")
s2_ll.align_zero_loss_peak(also_align=[s2])
s2.crop_signal(450., 590.)

Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	3.23
std:	0.249

min:	3
Q1:	3
median:	3
Q3:	3.5
max:	3.5


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

We will also check that the elements are correctly in [`metadata.Sample.elements`](https://hyperspy.org/exspy/user_guide/metadata_structure.html)

In [26]:
# Display the metadata
s2.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       └── collection_angle = np.float64(33.19)
│       ├── beam_energy = np.float64(80.0)
│       ├── convergence_angle = np.float64(27.42)
│       └── dwell_time = np.float64(0.1)
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 2.0
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2024-03-09T17:51:54.322801+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 2.0
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2024-03-09T17:52:04.954963+01:00
│   │   └── 2
│   │       ├── hyperspy_version = 2.4.0.dev3+g8ac7020d2
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2025-06-02T19:17:22.009211+01:00
│   ├── original_filename = EELS Spectrum Image (high-loss).dm3
│   └── title = EELS Spectrum Image (high-loss)
└── Signal
    ├── signal_origin = 
    └── signal_type = EELS

As no elements is defined in the metadata, we add them using the [`add_elements`](https://hyperspy.org/exspy/reference/signals.html#exspy.signals.EELSSpectrum.add_elements) method

In [27]:
s2.add_elements(("O", ))
s2.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       └── collection_angle = np.float64(33.19)
│       ├── beam_energy = np.float64(80.0)
│       ├── convergence_angle = np.float64(27.42)
│       └── dwell_time = np.float64(0.1)
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 2.0
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2024-03-09T17:51:54.322801+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 2.0
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2024-03-09T17:52:04.954963+01:00
│   │   └── 2
│   │       ├── hyperspy_version = 2.4.0.dev3+g8ac7020d2
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2025-06-02T19:17:22.009211+01:00
│   ├── original_filename = EELS Spectrum Image (high-loss).dm3
│   └── title = EELS Spectrum Image (high-loss)
├── Sample
│   └── elements = ['O']
└── Signal
    ├── signal_origin = 
    └── signal_type = EELS

We create the model. We pass the low-loss signal to take into account multiple scattering into the model: the model will be convolved with a the low-loss before being fitted to the data.

In [28]:
# Use the `create_model` method from the signal; don't forget to set the low loss
m2 = s2.create_model(low_loss=s2_ll)

We check that we have the expected components: a background the `O_K` edges 

In [29]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |            PowerLaw |            PowerLaw |            PowerLaw
   1 |                 O_K |                 O_K |          EELSCLEdge

We plot the model with the individually components as we will used these to observe how the fit proceed

In [30]:
m2.plot(True)

We perform a smart fit to fit components before fitting the peaks of the fine structure

In [31]:
m2.smart_fit()

As we can notice that the edge position doesn't describe well the data, we adjust the `onset_energy`. Visual inspecting shows us that the onset is at 524 eV. We want to do that for all navigation positions.

In [32]:
# Use the `set_parameters_value` method from the model
m2.set_parameters_value("onset_energy", 524, component_list=[1])

We run a fit again to check how the fit improves

In [33]:
m2.smart_fit()

Once we have satisfy with the current fit, fit all navigation positions

In [34]:
m2.multifit(kind="smart")

  0%|          | 0/40 [00:00<?, ?it/s]

We enable the fine structure and set the `fine_structure_width` to 30. Further documentation is available in the [eXSpy user guide](https://hyperspy.org/exspy/user_guide/eels.html#fine-structure-analysis-using-gaussian-functions).

In [35]:
m2.enable_fine_structure()
m2[1].fine_structure_width = 30

We fit all navigation positions.

In [36]:
m2.multifit(kind="smart")

  0%|          | 0/40 [00:00<?, ?it/s]

The fit looks very nice but this is still using spline to fit the fine structure. This approach of fitting the fine structure is useful to improve fit quality and get accurate edges intensity for quantification but it doesn't give use the characteristics (peaks, height, width) of the peaks of the fine structure.

## Add Gaussian to the fine structure

To get the characteristics (peaks, height, width) of the peaks of the fine structure, we will add `Gaussian` components and fit these in the same manner as done in the section 3.2.

In [37]:
# Create the three Gaussian copmponents
g1_m2 = hs.model.components1D.Gaussian()
g1_m2.name = "O_K_peak1"
g2_m2 = hs.model.components1D.Gaussian()
g2_m2.name = "O_K_peak2"
g3_m2 = hs.model.components1D.Gaussian()
g3_m2.name = "O_K_peak3"

In [38]:
# Set bounds on the `sigma` parameter
for g in [g1_m2, g2_m2, g3_m2]:
    g.sigma.bmin = 0.75
    g.sigma.bmax = 3.5

Add the `Gaussian` components to the list of `fine_structure_components`. We also set the `fine_structure_spline_onset`, which defines where the spline will start to a value of 18.

In [39]:
O_K = m2.components.O_K
O_K.fine_structure_components.update((g1_m2, g2_m2, g3_m2))
O_K.fine_structure_spline_onset = 18

Fit the three Gaussian components individually; observe how the fit progress before each steps and if it describes well the experimental data.

In [40]:
m2.fit_component(g2_m2, signal_range=(528., 532.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

In [41]:
m2.fit_component(g3_m2, signal_range=(535., 540.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

In [42]:
m2.fit_component(g1_m2, signal_range=(523., 526.), only_current=False, bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

In [43]:
m2.set_parameters_not_free(parameter_name_list=["centre"])

In [44]:
m2.smart_fit(bounded=True)

Finally, fit all components together for all navigation positions.

In [45]:
m2.multifit(kind="smart", bounded=True)

  0%|          | 0/40 [00:00<?, ?it/s]

# <a id='visualise_fitting_results'></a> 5. Visualise fitting results

We can then compare the parameters in the components, by getting the parameter values as signals using the [`as_signal`](https://hyperspy.org/hyperspy-doc/current/reference/base_classes/model/parameter.html#hyperspy.component.Parameter.as_signal) method.

Firstly get the ratio of the `A` parameters for the largest peak and the pre-peak, by using [`as_signal`](https://hyperspy.org/hyperspy-doc/current/reference/base_classes/model/parameter.html#hyperspy.component.Parameter.as_signal) and dividing the signals. Then plot the results.

## 5.1 Fitting after deconvolution

In [46]:
g1_g3_ratio = g1.A.as_signal() / g3.A.as_signal()

In [47]:
g1_g3_ratio.plot()

Then get the difference in centre positions between the largest peak and the pre-peak, via the `centre` parameter in the Gaussians.

In [48]:
g1_g3_position = g3.centre.as_signal() - g1.centre.as_signal()

In [49]:
g1_g3_position.plot()

Lastly, get the ratio of the `sigma` parameters in the largest peak and the pre-peak.

In [50]:
g1_g3_sigma = g1.sigma.as_signal() / g3.sigma.as_signal()

In [51]:
g1_g3_sigma.plot()

In all of the comparisons there are some large changes in the region with beam damage. However, the values can vary a great deal. This is most likely due to the pre-peak almost disappearing at some points in the line scan, leading to bad fitting of g1 and g3.

## 5.2 Fitting with convolution

In [52]:
g1_g3_ratio_m2 = g1_m2.A.as_signal() / g3_m2.A.as_signal()

In [53]:
g1_g3_ratio_m2.plot()

Then get the difference in centre positions between the largest peak and the pre-peak, via the `centre` parameter in the Gaussians.

In [54]:
g1_g3_position_m2 = g3_m2.centre.as_signal() - g1_m2.centre.as_signal()

In [55]:
g1_g3_position_m2.plot()

Lastly, get the ratio of the `sigma` parameters in the largest peak and the pre-peak.

In [56]:
g1_g3_sigma_m2 = g1_m2.sigma.as_signal() / g3_m2.sigma.as_signal()

In [57]:
g1_g3_sigma_m2.plot()

## 5.3 Comparison of fitting results for both approaches

In [60]:
ax = hs.plot.plot_spectra([g1_g3_ratio, g1_g3_ratio_m2], legend=["Deconvolution approach", "Convolution approach"])
ax.set_ylabel("1st to 3rd peak intensity ratio")

Text(0, 0.5, '1st to 3rd peak intensity ratio')

In [62]:
ax = hs.plot.plot_spectra([g1_g3_position, g1_g3_position_m2], legend=["Deconvolution approach", "Convolution approach"])
ax.set_ylabel("1st to 3rd peak separation (eV)")

Text(0, 0.5, '1st to 3rd peak separation (eV)')